In [25]:
import pandas as pd
import requests
import time  # Importar el módulo time

# Cargar el archivo .pkl
aeropuertos_df = pd.read_pickle('C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/aeropuertos_unicos.pkl')

# Función para buscar información en Wikipedia usando la función de búsqueda
def buscar_en_wikipedia(titulo):
    S = requests.Session()
    URL = "https://es.wikipedia.org/w/api.php"

    PARAMS = {
        "action": "query",
        "list": "search",
        "srsearch": titulo,
        "format": "json",
        "srlimit": 1
    }

    response = S.get(url=URL, params=PARAMS)
    data = response.json()

    search_results = data['query']['search']
    if search_results:
        page_title = search_results[0]['title']
        # Ahora obtenemos el extracto de la página seleccionada
        EXTRACT_PARAMS = {
            "action": "query",
            "prop": "extracts",
            "exintro": True,
            "explaintext": True,
            "titles": page_title,
            "format": "json"
        }
        extract_response = S.get(url=URL, params=EXTRACT_PARAMS)
        extract_data = extract_response.json()
        page = next(iter(extract_data['query']['pages'].values()))
        if 'extract' in page:
            return page['extract']
    return 'No se encontró información.'

# Aplicar la función de búsqueda a cada aeropuerto y almacenar el resultado en la nueva columna 'wiki_info'
def aplicar_busqueda_y_pausa(nombre):
    resultado = buscar_en_wikipedia("aeropuerto " + nombre)
    time.sleep(1)  # Hace una pausa después de cada solicitud
    return resultado

aeropuertos_df['wiki_info'] = aeropuertos_df['nombre_aeropuerto'].apply(aplicar_busqueda_y_pausa)

# Mostrar los primeros resultados para verificar
print(aeropuertos_df.head())


                          nombre_aeropuerto codigo_aeropuerto  \
0                          Abilene Regional               ABI   
1                     Akron-Canton Regional               CAK   
2  Hartsfield-Jackson Atlanta International               ATL   
3  Jackson Medgar Wiley Evers International               JAN   
4                Jacksonville International               JAX   

              ciudad estado   latitude  longitude  \
0            Abilene     TX   32.45095 -99.735678   
1              Akron     OH  41.080564 -81.519677   
2            Atlanta     GA  33.640846 -84.432594   
3  Jackson/Vicksburg     MS  32.353325 -90.880255   
4       Jacksonville     FL  30.328992 -81.657959   

                                           direccion  \
0        202 Cedar St (North 2nd), Abilene, TX 79601   
1                     182 S Main St, Akron, OH 44308   
2  6000 N Terminal Pkwy (at Airport Loop Rd), Atl...   
3                 923 Walnut St, Vicksburg, MS 39183   
4  303 N L

In [26]:
# Contar cuántos NaN hay en la columna 'wiki_info'
cantidad_nan = aeropuertos_df['wiki_info'].isna().sum()
print(f"Cantidad de NaN en 'wiki_info': {cantidad_nan}")


Cantidad de NaN en 'wiki_info': 0


In [27]:
aeropuertos_df.to_pickle('C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/aeropuertos_unicos.pkl')

In [19]:
def obtener_secciones_wikipedia(titulo):
    S = requests.Session()
    URL = "https://es.wikipedia.org/w/api.php"

    # Obtener las secciones de la página
    SECCIONES_PARAMS = {
        "action": "parse",
        "page": titulo,
        "format": "json",
        "prop": "sections",
    }

    secciones_response = S.get(url=URL, params=SECCIONES_PARAMS)
    secciones_data = secciones_response.json()

    # Lista para almacenar los títulos de las secciones
    titulos_secciones = []
    if 'parse' in secciones_data:
        secciones = secciones_data['parse']['sections']
        for seccion in secciones:
            titulos_secciones.append((seccion['index'], seccion['line']))
    return titulos_secciones

# Buscar primero el título de la página con el método anterior
titulo_pagina = buscar_en_wikipedia(nombre_aeropuerto_ejemplo)  # Asegúrate de adaptar esta línea para que devuelva solo el título

# Obtener las secciones de la página
secciones = obtener_secciones_wikipedia(titulo_pagina)
print(secciones)


[]


In [20]:
def obtener_wikitext(titulo):
    S = requests.Session()
    URL = "https://es.wikipedia.org/w/api.php"

    PARAMS = {
        "action": "query",
        "titles": titulo,
        "prop": "revisions",
        "rvprop": "content",
        "format": "json",
        "rvslots": "main"
    }

    response = S.get(url=URL, params=PARAMS)
    data = response.json()

    pages = data['query']['pages']
    page = next(iter(pages.values()))  # Obtener la primera página
    if 'revisions' in page:
        wikitext = page['revisions'][0]['slots']['main']['*']
        return wikitext
    return None

# Obtener el wikitext de la página de Wikipedia
wikitext = obtener_wikitext(titulo_pagina)  # Asegúrate de usar el título correcto de la página


In [23]:
print(wikitext)

None


In [21]:
import mwparserfromhell

# Parsear el wikitext
parsed_wikitext = mwparserfromhell.parse(wikitext)

# Buscar los templates del infobox
templates = parsed_wikitext.filter_templates()

# Este es un ejemplo muy básico y probablemente necesitarás ajustarlo
for template in templates:
    if 'infobox' in template.name.lower():
        for param in template.params:
            print(param.name.strip(), param.value.strip())


ModuleNotFoundError: No module named 'mwparserfromhell'

In [11]:
info_aeropuerto

'Jacksonville International Airport (IATA: JAX, ICAO: KJAX, FAA LID: JAX) is a civil-military public airport 13 miles (21 km) north of Downtown Jacksonville, in Duval County, Florida. It is owned and operated by the Jacksonville Aviation Authority.'

In [18]:
import pandas as pd
import requests

# Cargar el archivo .pkl
aeropuertos_df = pd.read_pickle('C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/aeropuertos_unicos.pkl')

# Selecciona un aeropuerto de ejemplo y añade la palabra "aeropuerto" al final
nombre_aeropuerto_ejemplo = "aeropuerto " + aeropuertos_df.iloc[4]['nombre_aeropuerto']

def buscar_info_completa_wikipedia(titulo):
    S = requests.Session()
    URL = "https://es.wikipedia.org/w/api.php"

    # Primero, busca el título de la página usando la función de búsqueda
    SEARCH_PARAMS = {
        "action": "query",
        "list": "search",
        "srsearch": titulo,
        "format": "json",
        "srlimit": 1
    }

    search_response = S.get(url=URL, params=SEARCH_PARAMS)
    search_data = search_response.json()

    search_results = search_data['query']['search']
    if search_results:
        page_title = search_results[0]['title']

        # Luego, obtén el contenido completo de la página encontrada
        CONTENT_PARAMS = {
            "action": "parse",
            "page": page_title,
            "format": "json",
            "prop": "text"  # Obtener el texto completo de la página
        }

        content_response = S.get(url=URL, params=CONTENT_PARAMS)
        content_data = content_response.json()

        if 'parse' in content_data:
            text = content_data['parse']['text']['*']
            return text  # Esto devolverá HTML
    return 'No se encontró información.'

# Usar el título del artículo obtenido anteriormente para obtener información completa
info_completa_aeropuerto = buscar_info_completa_wikipedia(nombre_aeropuerto_ejemplo)
print(info_completa_aeropuerto)



<div class="mw-content-ltr mw-parser-output" lang="es" dir="ltr"><style data-mw-deduplicate="TemplateStyles:r157776694">.mw-parser-output .infobox .imagen{max-width:100%;margin:0 auto}.mw-parser-output .infobox .imagen img{max-width:100%;height:auto}.mw-parser-output .infobox .mw-kartographer-container .thumbinner,.mw-parser-output .infobox .mw-kartographer-map{box-sizing:border-box;width:100%!important}body.skin-timeless .mw-parser-output .infobox .imagen a.image>img{max-width:100%!important;height:auto!important}</style><table class="infobox" style="width:22.7em; line-height: 1.4em; text-align:left; padding:.23em;"><tbody><tr><th colspan="3" class="cabecera" style="text-align:center;background-color:transparent;color:inherit;">Aeropuerto Internacional de Jacksonville</th></tr><tr><td colspan="3" style="text-align:center;background-color:#5FA5EB; color:white; font-size: 90%; font-weight: bold;;">
Jacksonville International Airport</td></tr><tr><td colspan="3" class="imagen" style="tex

In [16]:
info_completa_aeropuerto

'No se encontró información.'